In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import xgboost as xgb
import cupy as cp
from sklearn.decomposition import PCA
from xgboost import plot_importance
import wandb
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

In [6]:
#Sadece deamnd verisi kullanma
# demand_array=df["total_demand"]
# n_steps=168
# n_forecast=24
# X, y = [], []
# for i in range(len(demand_array)):
#     # find the end of this pattern
#     end_ix = i + n_steps
#     # check if we are beyond the sequence
#     if end_ix > len(demand_array) - n_forecast:
#         break
#     # gather input and output parts of the pattern
#     seq_x = demand_array[i:end_ix].values
#     seq_y = demand_array[end_ix:n_forecast+end_ix].values
#     X.append(seq_x)
#     y.append(seq_y)
# X=np.array(X)
# y=np.array(y)


# # Calculate the index to split the data
# split_index = int(len(X) * 2 / 3)

# # Split the data into training and validation sets
# X_train = X[:split_index]
# y_train = y[:split_index]
# X_test = X[split_index:]
# y_test = y[split_index:]

In [4]:
df=pd.read_csv("df_xgboost.csv",delimiter=";")

In [5]:
df.columns

Index(['AIR_TEMP_MIN', 'PRCP', 'total_demand', 'day_of_week_Monday',
       'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday',
       'day_of_week_Tuesday', 'day_of_week_Wednesday', 'week_of_year',
       'is_special_day', 'season_autumn', 'season_spring', 'season_winter',
       'hour_of_day'],
      dtype='object')

In [6]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
def generate_lagged_features(df, n_step,n_next):
    """
    Generate lagged features for the demand based on the n_step value and store the next 24 demand values as an array.

    Args:
    - df (pd.DataFrame): Original dataframe
    - n_step (int): Number of hours to consider for previous demand

    Returns:
    - df_new (pd.DataFrame): Dataframe with new features
    """

    # Creating lag features for demand
    for i in range(1, n_step + 1):
        df[f'demand_lag_{i}'] = df['total_demand'].shift(i)
    


    # Dropping rows with NaN values
    df_new = df.dropna()


    # Resetting index
    df_new = df_new.reset_index(drop=True)
    # Store the next 24 demand values as an array for the rows that will remain in the DataFrame
    next_demand_values = []
    # Calculate the next 24 demand values for each row
    for i in range(len(df_new) - n_next):
        next_demand_values.append(df_new['total_demand'].tolist()[i+1:i+n_next+1])
    print(np.array(next_demand_values).shape)
    df_new = df_new.iloc[:-n_next]
    # Add the next_24_demand values as a new column
    df_new['total_demand_'] = next_demand_values

    # Reshape the data for regression modeling
    df_test_ = pd.DataFrame(df_new['total_demand_'].tolist(), columns=[f'demand_{i+1}_hour' for i in range(n_next)])
    df_new = df_new.drop('total_demand_', axis=1)

    # Resetting index
    df_new = df_new.reset_index(drop=True)

    
    return df_new,df_test_

def minmax_scale_dataframe(data):
    scaler = MinMaxScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
    return df_scaled


In [7]:
df_X,df_y = generate_lagged_features(df, n_step = 168,n_next=8)


C:\Users\murca\AppData\Local\Temp\ipykernel_1748\2553173045.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'demand_lag_{i}'] = df['total_demand'].shift(i)
C:\Users\murca\AppData\Local\Temp\ipykernel_1748\2553173045.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'demand_lag_{i}'] = df['total_demand'].shift(i)
C:\Users\murca\AppData\Local\Temp\ipykernel_1748\2553173045.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

(26128, 8)


In [11]:
#Special Day ekleme


# # Ensure valid_start column is of type datetime
# df_new['valid_start'] = pd.to_datetime(df_new['valid_start'])

# # Extract the day of the week
# df_new['day_of_week'] = df_new['valid_start'].dt.day_name()

# # One-hot encode the day_of_week column with only 6 dummy columns
# df_encoded = pd.get_dummies(df_new, columns=['day_of_week'], drop_first=True)
# df_encoded['week_of_year'] = df_encoded['valid_start'].dt.isocalendar().week
# # Define the special holidays
# special_holidays = [
#     "1-1",  # New Year's Day
#     "2-1",  # New Year's Day (additional day)
#     "26-1",  # Australia Day
#     "13-3",  # Labour Day
#     "7-4",  # Good Friday
#     "8-4",  # Easter Saturday
#     # Since you did not provide a date for "Easter Sunday", I'm omitting it.
#     "10-4",  # Easter Monday
#     "25-4",  # Anzac Day
#     "12-6",  # King's Birthday
#     # Omitting the AFL Grand Final date since it's TBC.
#     "7-11",  # Melbourne Cup
#     "25-12",  # Christmas Day
#     "26-12"  # Boxing Day
# ]
# def format_date(date):
#     formatted = date.strftime("%d-%m")
#     day, month = formatted.split('-')
#     return f"{int(day)}-{int(month)}"

# df_encoded['is_special_day'] = df_encoded['valid_start'].apply(lambda x: 1 if format_date(x) in special_holidays else 0)
# def get_season(date):
#     month = date.month
#     if 12 <= month <= 2:
#         return 'summer'
#     elif 3 <= month <= 5:
#         return 'autumn'
#     elif 6 <= month <= 8:
#         return 'winter'
#     elif 9 <= month <= 11:
#         return 'spring'
# # Add a 'season' column to the dataframe
# df_encoded['season'] = df_encoded['valid_start'].apply(get_season)

# # One-hot encode the 'season' column
# df_encoded = pd.get_dummies(df_encoded, columns=['season'], prefix='season')
# # Extract hour from the 'valid_start' column
# df_encoded['hour_of_day'] = df_encoded['valid_start'].dt.hour

# # Create a 'season_label' column to convert one-hot encoding to a single column of season labels
# conditions = [
#     df_encoded['season_autumn'] == 1,
#     df_encoded['season_spring'] == 1,
#     df_encoded['season_winter'] == 1
# ]

# choices = ['Autumn', 'Spring', 'Winter']

# df_encoded['season_label'] = np.select(conditions, choices, default='Summer')

# df_encoded['year'] = df_encoded['valid_start'].dt.year
# days_columns = ['day_of_week_Monday', 'day_of_week_Tuesday', 'day_of_week_Wednesday', 'day_of_week_Thursday', 'day_of_week_Friday', 'day_of_week_Saturday', 'day_of_week_Sunday']

# df_encoded['day_name'] = df_encoded[days_columns].idxmax(axis=1).str.split('_').str[-1]
# # Extract just the date from the valid_start column
# df_encoded['date'] = df_encoded['valid_start'].dt.date

# def get_label_from_one_hot(row, prefix, categories):
#     """ This function will return the label by checking which column is set to 1 for one-hot encoded columns. 
#         If all are 0, return the dropped category, assumed to be the first in the provided categories list. """
#     for col in row.index:
#         if col.startswith(prefix) and row[col] == 1:
#             return col.split('_')[-1]
#     return categories[0]  # Assuming the first category is the one dropped

# # List of original categories before creating dummy variables
# original_day_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
# original_season_categories = ['spring', 'summer', 'autumn', 'winter']

# # Apply the function to each row for day_of_week and season
# df_new['day_of_week_label'] = df_new.apply(lambda row: get_label_from_one_hot(row, 'day_of_week', original_day_categories), axis=1)
# df_new['season_label'] = df_new.apply(lambda row: get_label_from_one_hot(row, 'season', original_season_categories), axis=1)

In [8]:
from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
split_val_index = int(len(df_X) * 2 / 3)
split_test_index = int(len(df_X) * 5 / 6)
# Separate independent and dependent variables for training set
X_train = df_X.loc[:split_val_index].values
y_train = df_y.loc[:split_val_index].values

# Separate independent and dependent variables for validation set
X_val = df_X.loc[split_val_index:split_test_index].values
y_val = df_y.loc[split_val_index:split_test_index].values

X_test = df_X.loc[split_test_index:].values
y_test = df_y.loc[split_test_index:].values


scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train=scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
X_val = scaler_X.transform(X_val)

In [13]:

# pca = PCA(n_components=0.99)
# pca.fit(X_train)
# X_train = pca.transform(X_train)
# X_test = pca.transform(X_test)

In [9]:
X_train.shape

(17419, 183)

In [92]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

0

In [10]:
wandb.init(project="DI502_PROJECT")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: muratcankaplann98. Use `wandb login --relogin` to force relogin


In [13]:
sweep_config = {
    'method': 'bayes',  # Use Bayesian optimization
    'metric': {
        'name': 'loss',
        'goal': 'minimize'   
    },
    'parameters': {
        'booster': {
            'value': 'gbtree'
        },
        'objective': {
            'value': 'reg:squarederror'
        },
        'colsample_bytree': {
            'min': 0.1,
            'max': 1.0
        },
        'learning_rate': {
            'min': 0.01,
            'max': 0.5
        },
        'max_depth': {
            'min': 3,
            'max': 10
        },
        'sampling_method': {
            'value': 'gradient_based'
        },
        'alpha': {
            'min': 0,
            'max': 100
        },
        'n_estimators': {
            'values': [100, 200, 300, 400, 500]
        },
        'grow_policy': {
            'value': 'lossguide'
        },
    }
    # No need to include 'stop' for Bayesian optimization
    # But you can still include it if you want to limit the number of runs
}



In [14]:
def train():
    with wandb.init() as run:
        config = run.config

        # Initialize the XGBoost model with the specified parameters from the sweep
        model = xgb.XGBRegressor(
            booster=config.booster,
            objective=config.objective,
            colsample_bytree=config.colsample_bytree,
            learning_rate=config.learning_rate,
            max_depth=int(config.max_depth),
            sampling_method=config.sampling_method,
            alpha=config.alpha,
            n_estimators=config.n_estimators,
            grow_policy=config.grow_policy,
            tree_method='gpu_hist',  # Ensures computation is done on GPU
            predictor='gpu_predictor'  # Ensures prediction is done on GPU
        )
        
        # Fit the model (no need to convert to Cupy, XGBoost handles GPU operations)
        eval_set = [(X_train, y_train), (X_val, y_val)]
        model.fit(X_train, y_train, eval_set=eval_set, verbose=True)
        
        # Evaluate the model
        mse_train, mape_train = evaluate_model(model, X_train, y_train)
        mse_val, mape_val = evaluate_model(model, X_val, y_val)
        mse_test, mape_test = evaluate_model(model, X_test, y_test)
        
        # Log the metrics to wandb
        wandb.log({
            "mse_train": mse_train, "mape_train": mape_train,
            "mse_val": mse_val, "mape_val": mape_val,
            "mse_test": mse_test, "mape_test": mape_test,
            "loss": mape_test  # This is the metric we want to minimize
        })


In [15]:
def evaluate_model(model, X, y):
    # Make predictions on the provided dataset
    predictions = model.predict(X)

    # Calculate MSE and MAPE
    mse = mean_squared_error(y, predictions)
    mape = mean_absolute_percentage_error(y, predictions)

    return mse, mape

In [16]:
sweep_id = wandb.sweep(sweep_config, project='DI502_PROJECT')
wandb.agent(sweep_id, train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: hg5769yf
Sweep URL: https://wandb.ai/muratcankaplann98/DI502_PROJECT/sweeps/hg5769yf


wandb: Agent Starting Run: 8m7m6s5z with config:
wandb: 	alpha: 70
wandb: 	booster: gbtree
wandb: 	colsample_bytree: 0.66627973266956
wandb: 	grow_policy: lossguide
wandb: 	learning_rate: 0.031448702074092796
wandb: 	max_depth: 6
wandb: 	n_estimators: 200
wandb: 	objective: reg:squarederror
wandb: 	sampling_method: gradient_based


[0]	validation_0-rmse:1727.33006	validation_1-rmse:1605.73727
[1]	validation_0-rmse:1683.15846	validation_1-rmse:1564.00869
[2]	validation_0-rmse:1640.58223	validation_1-rmse:1523.88442
[3]	validation_0-rmse:1599.41830	validation_1-rmse:1485.23228
[4]	validation_0-rmse:1559.37330	validation_1-rmse:1447.26820
[5]	validation_0-rmse:1520.42111	validation_1-rmse:1410.73496
[6]	validation_0-rmse:1482.81030	validation_1-rmse:1375.53755
[7]	validation_0-rmse:1446.71093	validation_1-rmse:1341.19390
[8]	validation_0-rmse:1411.23336	validation_1-rmse:1308.05044
[9]	validation_0-rmse:1377.08236	validation_1-rmse:1275.87240
[10]	validation_0-rmse:1344.54045	validation_1-rmse:1245.53050
[11]	validation_0-rmse:1313.16844	validation_1-rmse:1216.06380
[12]	validation_0-rmse:1282.77571	validation_1-rmse:1187.78163
[13]	validation_0-rmse:1252.64999	validation_1-rmse:1159.60051
[14]	validation_0-rmse:1224.37124	validation_1-rmse:1133.08837
[15]	validation_0-rmse:1197.05768	validation_1-rmse:1107.37364
[1

loss,▁
mape_test,▁
mape_train,▁
mape_val,▁
mse_test,▁
mse_train,▁
mse_val,▁
loss,0.03736
mape_test,0.03736
mape_train,0.02267
mape_val,0.02845


...

[0]	validation_0-rmse:1551.49934	validation_1-rmse:1433.96305
[1]	validation_0-rmse:1373.31996	validation_1-rmse:1260.43344
[2]	validation_0-rmse:1227.33048	validation_1-rmse:1121.20034
[3]	validation_0-rmse:1109.33848	validation_1-rmse:1005.20433
[4]	validation_0-rmse:1012.59880	validation_1-rmse:910.07075
[5]	validation_0-rmse:932.43748	validation_1-rmse:836.16115
[6]	validation_0-rmse:868.41772	validation_1-rmse:773.67799
[7]	validation_0-rmse:814.87350	validation_1-rmse:724.78053
[8]	validation_0-rmse:769.71674	validation_1-rmse:685.21295
[9]	validation_0-rmse:733.98373	validation_1-rmse:651.35253
[10]	validation_0-rmse:704.95474	validation_1-rmse:626.16753
[11]	validation_0-rmse:679.97499	validation_1-rmse:602.60381
[12]	validation_0-rmse:657.31828	validation_1-rmse:585.46968
[13]	validation_0-rmse:637.02123	validation_1-rmse:571.23273
[14]	validation_0-rmse:620.11354	validation_1-rmse:557.80961
[15]	validation_0-rmse:605.03581	validation_1-rmse:546.64741
[16]	validation_0-rmse:59

loss,▁
mape_test,▁
mape_train,▁
mape_val,▁
mse_test,▁
mse_train,▁
mse_val,▁
loss,0.03512
mape_test,0.03512
mape_train,0.01419
mape_val,0.02772


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


Error in callback <function _WandbInit._pause_backend at 0x00000293D2AEA320> (for post_run_cell):


ConnectionResetError: [WinError 10054] Varolan bir bağlantı uzaktaki bir ana bilgisayar tarafından zorla kapatıldı

In [25]:
# import joblib
# joblib.dump(model, "4_weeks_to_1_weeks_pred_model.pkl")
# import joblib
# loaded_model = joblib.load("4_weeks_to_1_weeks_pred_model.pkl")
# loaded_model.fit(X_train, y_train)

# # Make predictions for the next 8 hours
# y_pred = loaded_model.predict(X_test)

['4_weeks_to_1_weeks_pred_model.pkl']

In [78]:
#Veriseti yazma

# y_pred = model.predict(df_X.drop(columns=['valid_start']))
# y_pred=pd.DataFrame(y_pred,columns=df_y.columns)
# df_1_week_24_pred=pd.concat([df_X[['valid_start', 'AIR_TEMP', 'AIR_TEMP_MAX', 'AIR_TEMP_MIN', 'PRCP',"total_demand"]], y_pred], axis=1)
# df_1_week_24_pred.to_csv("df_1_week_24_pred.csv")

In [11]:
from sklearn.metrics import mean_squared_error
y_pred_test = scaler_y.inverse_transform(model.predict(cp.array(X_test)))
y_pred_val = scaler_y.inverse_transform(model.predict(cp.array(X_val)))
def calculate_mape(actual, predicted):
    return (1 / len(actual)) * sum(abs((actual - predicted) / actual) * 100)

# Evaluate the model
mape_test = calculate_mape(y_test, y_pred_test)
mape_val = calculate_mape(y_val, y_pred_val)
mse_test = mean_squared_error(y_test, y_pred_test)
mse_val = mean_squared_error(y_val, y_pred_val)
print("8 hours prediction Mean Absolute Percentage Error for Test Dataset:", mape_test.mean())
print("8 hours prediction Mean Absolute Percentage Error for Validation Dataset:", mape_val.mean())
print("8 hours prediction Mean Squared Error for Test Dataset:", np.sqrt(mse_test.mean()))
print("8 hours prediction Mean Squared Error for Validation Dataset:", np.sqrt(mse_val.mean()))

8 hours prediction Mean Absolute Percentage Error for Test Dataset: 3.8605188944717885
8 hours prediction Mean Absolute Percentage Error for Validation Dataset: 2.975193504426903
8 hours prediction Mean Squared Error for Test Dataset: 568.9029699927256
8 hours prediction Mean Squared Error for Validation Dataset: 414.57825039005263


In [25]:
(mape_test.mean()+mape_val.mean())*0.5

3.4178561994493455

In [96]:
indice = 38  # Adjust the index as needed

# Set the title
plt.title("Daily Prediction")

# Plot the actual and predicted values
plt.plot(np.array(y_test[indice]), label="Actual")
plt.plot(np.array(y_pred[indice]), label="Predicted")

# Add labels to the x and y axes
plt.xlabel("Time Steps")
plt.ylabel("Demand Value")

# Add a legend to distinguish between actual and predicted lines
plt.legend()

# Show the plot
plt.show()

In [97]:
fi = pd.DataFrame(data=model.feature_importances_, index=df_X.columns, columns=['importance'])
fi = fi.sort_values(by='importance', ascending=False)

# Plot the top 10 features
top_n = 10
fi[:top_n].sort_values('importance').plot(kind='barh', title=f'Top {top_n} Feature Importance')
plt.show()